In [112]:
!pip install rpy2==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
import numpy as np
import scipy as sp
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Bayesian/Progetto/ARIMA_STAN_matrices/data.csv')
df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df.set_index('Date', inplace=True)
df.fillna(np.NaN, inplace=True)


import rpy2.rinterface
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [115]:
import seaborn as sns
import matplotlib.pyplot as plt

In [96]:
df


,CASTELLUCCIO,FEBBIO,CORTE BRUGNATELLA,SAVIGNANO DI RIGO,SAN LEO,VERUCCHIO,BADIA,GIARDINI MARGHERITA,PARCO RESISTENZA,PARCO BERTOZZI,...,S. LAZZARO,S. ROCCO,MALCANTONE,FLAMINIA,GIARDINI,MONTEBELLO,CITTADELLA,SAN FRANCESCO,GERBIDO,TIMAVO
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0.602060,0.778151,1.000000,1.079181,1.113943,1.397940,1.591065,1.633468,1.707570,1.643453,...,1.681241,1.653213,1.643453,1.653213,1.785330,1.707570,1.724276,NaN,1.672098,1.792392
2018-01-02,0.477121,0.477121,0.602060,0.477121,0.845098,1.079181,1.113943,1.255273,1.447158,1.342423,...,1.342423,1.397940,1.414973,1.556303,1.414973,1.544068,1.397940,NaN,1.431364,1.602060
2018-01-03,NaN,0.698970,0.698970,0.301030,NaN,0.778151,1.079181,1.204120,1.230449,1.278754,...,1.414973,1.447158,1.397940,1.431364,1.477121,1.397940,1.342423,NaN,1.568202,1.556303
2018-01-04,NaN,0.301030,0.477121,0.698970,0.698970,1.322219,1.176091,1.361728,1.591065,1.568202,...,1.531479,1.544068,1.477121,NaN,1.623249,1.568202,1.462398,1.556303,1.579784,1.568202
2018-01-05,0.602060,0.698970,1.113943,0.845098,1.113943,1.653213,1.531479,1.633468,1.763428,1.707570,...,1.716003,1.602060,1.591065,1.851258,1.792392,1.716003,1.681241,1.755875,1.698970,1.724276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,0.903090,0.778151,0.903090,0.903090,NaN,NaN,1.278754,NaN,1.361728,1.322219,...,1.380211,1.477121,1.431364,1.477121,1.447158,1.579784,1.531479,1.643453,1.612784,1.531479
2018-12-28,0.698970,0.778151,0.778151,0.845098,NaN,NaN,1.204120,NaN,1.591065,1.556303,...,1.643453,1.681241,1.633468,1.716003,1.770852,1.698970,1.653213,1.518514,1.755875,1.755875
2018-12-29,0.903090,1.079181,0.602060,1.000000,1.079181,1.380211,1.146128,NaN,1.633468,1.623249,...,1.681241,1.732394,1.690196,1.707570,1.755875,1.681241,1.681241,1.591065,1.770852,1.748188


In [116]:
%%R

install.packages("bsts")
library(bsts)


(as ‘lib’ is unspecified)







	‘/tmp/RtmpE7tpKD/downloaded_packages’



In [117]:
%R -i df

In [118]:
%%R
#stazione=1
posterior=array(rep(0,49*3000*365), dim=c(49,3000,1,365))

for(stazione in 1:49){
  staz<-df[,stazione]
  ss = AddLocalLinearTrend(list(),staz)
  bsts.model <- bsts(staz, state.specification = ss, niter = 3000, ping=0, seed=2016)
  aux=bsts.model[["state.contributions"]]
  posterior[stazione,,,]=aux
}


In [106]:
%R -o posterior

In [107]:
posterio=np.reshape(posterior,(posterior.shape[0],posterior.shape[1],posterior.shape[3]))

In [109]:
y_post_pred=np.moveaxis(posterio,0,-1)


## Grafici

In [110]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mplt
import seaborn as sns
import arviz as az
import datetime

In [111]:
@widgets.interact(stazione = df.columns)
def f(stazione):
    col_map = sns.light_palette((20, 75, 70), input='husl', as_cmap=True)

    plt.figure(figsize=(14,8))
    ax = plt.subplot(1,1,1)

    idx_stazione = df.columns.to_list().index(stazione)
    
    """
    sns.lineplot(np.transpose(y_post_pred[0:50,:,idx_stazione]))
    """

    lower_lim = np.zeros(len(df.index))
    upper_lim = np.zeros(len(df.index))    
    for i in range(len(df.index)):
        lower_lim[i] = np.percentile(y_post_pred[:,i,idx_stazione],2.5)
        upper_lim[i] = np.percentile(y_post_pred[:,i,idx_stazione],97.5)
    
    
#    sns.lineplot(lower_lim)
#    sns.lineplot(upper_lim)
    
    for i in range(len(df.index)):
        ax.add_patch(mplt.patches.Rectangle((i,lower_lim[i]),1,upper_lim[i]-lower_lim[i], fill=True, color=col_map(180)))

    sns.lineplot(data=df[stazione])    
    sns.lineplot(data=np.mean(y_post_pred[:,:,idx_stazione], axis=0))


    index = 0
    for line in ax.get_lines():
        if index == 0:
            col_map = sns.dark_palette((230,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(155))
        else:
            col_map = sns.dark_palette((10,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
        index += 1
        
    plt.ylim(bottom=0,top=2.5)

    primo_giorno = datetime.date(2018,1,1)
    date_da_segnare = []
    date_da_segnare_posizioni = []

    for i in range(12):
        date_da_segnare.append(datetime.date(2018,i+1,1))
        date_da_segnare_posizioni.append((date_da_segnare[2*i] - primo_giorno).days)
        date_da_segnare.append(datetime.date(2018,i+1,15))
        date_da_segnare_posizioni.append((date_da_segnare[2*i+1] - primo_giorno).days)
        date_da_segnare[2*i] = date_da_segnare[2*i].isoformat()
        date_da_segnare[2*i+1] = date_da_segnare[2*i+1].isoformat()

    plt.xticks(date_da_segnare_posizioni,date_da_segnare,rotation=65)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True)
    plt.grid()
    
    """
    ax.get_legend().remove()
    col_vals = np.linspace(1,255,num=len(df.columns))
    index = 0
    for line in ax.get_lines():
        if(index == len(ax.get_lines()) - 1):
            col_map = sns.dark_palette((230,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
            continue
        if(index == len(ax.get_lines()) - 2):
            col_map = sns.dark_palette((120,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
            index += 1
            continue

        #line.set_c(col_map(int(np.round(col_vals[index]))))
        line.set_c(col_map(220))
        line.set_alpha(0.3)
        index += 1
    """
    
    

    plt.show()



interactive(children=(Dropdown(description='stazione', options=('CASTELLUCCIO', 'FEBBIO', 'CORTE BRUGNATELLA',…